## Notebook 3: Modelado y Evaluación

**Objetivo:** Cargar los subconjuntos de datos para ambas vertientes (unidades y guaraníes), entrenar los modelos (Lineal, Cuadrático, KNN), evaluar su rendimiento y guardar todos los resultados.

**Fases:**
1.  **Configuración:** Importar librerías y funciones.
2.  **Bucle de Ejecución:** Iterar sobre 'cantidad' y 'monto_venta'.
    a. Cargar los datasets correspondientes.
    b. Ejecutar los experimentos.
    c. Guardar los resultados en subcarpetas.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

# --- Rutas ---
BASE_DIR = '..'
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, '02_Data', '02_processed')
RAW_DATA_DIR = os.path.join(BASE_DIR, '02_Data', '01_raw')
RESULTS_DIR = os.path.join(BASE_DIR, '04_Resultados')
TABLES_DIR = os.path.join(RESULTS_DIR, '01_Tablas_Detalladas')

# --- Parámetros de Ejecución ---
target_variable = 'all' # 'cantidad', 'monto_venta' o 'all'
datasets_to_run = ['F_48_NP'] # Default actualizado
models_to_run = ['Lineal', 'Cuadrático', 'KNN', 'SARIMA']
metrics_to_calculate = ['R2', 'MAE', 'RMSE', 'MASE', 'ME']

# --- ¡NUEVO PARÁMETRO! ---
maxiter = 50 # Valor por defecto para auto_arima (Papermill lo sobrescribirá)

In [3]:
import pandas as pd
import os
import sys
import logging 

# Importar las funciones del 'motor' (utils.py)
try:
    from utils.utils import ejecutar_experimentos, cargar_datos_dbf
except ImportError:
    print("Error: No se pudieron importar las funciones desde 'utils.utils'.")
    print("Asegúrate de que 03_Code/utils/utils.py existe.")

os.makedirs(TABLES_DIR, exist_ok=True)

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

print("Librerías y módulos importados. Rutas definidas.")

Librerías y módulos importados. Rutas definidas.


In [4]:
# ### 2. Bucle de Ejecución para Ambas Vertientes

# Determinar los targets a procesar basado en el parámetro
if isinstance(target_variable, list):
    targets_a_predecir = target_variable
elif target_variable == 'all':
    targets_a_predecir = ['cantidad', 'monto_venta']
else:
    targets_a_predecir = [target_variable] 

try:
    # Cargar información de productos una sola vez
    _, _, df_info_productos = cargar_datos_dbf(RAW_DATA_DIR)
    df_info_productos = df_info_productos[['CODI', 'DES']].rename(columns={'CODI': 'codigo_producto', 'DES': 'descripcion_producto'})
    df_info_productos['codigo_producto'] = df_info_productos['codigo_producto'].str.strip()
except Exception as e:
    print(f"Error al cargar 'df_info_productos' de DBF: {e}")
    df_info_productos = pd.DataFrame(columns=['codigo_producto', 'descripcion_producto']) 


for target in targets_a_predecir:
    print(f"\n{'='*20} INICIANDO MODELADO (CON CV) PARA: {target.upper()} {'='*20}")
    
    # Definir carpetas de entrada y salida
    input_subdir = os.path.join(PROCESSED_DATA_DIR, target)
    output_subdir = os.path.join(TABLES_DIR, target)
    os.makedirs(output_subdir, exist_ok=True)
    
    # --- Carga de Datos --- (Usando los datasets del parámetro)
    print(f"\nCargando subconjuntos de datos para modelar '{target}'...")
    nombres_datasets = datasets_to_run
    datasets_para_modelar = {}

    for nombre in nombres_datasets:
        try:
            path = os.path.join(input_subdir, f'ventas_{nombre}.csv')
            datasets_para_modelar[nombre] = pd.read_csv(path, parse_dates=['mes'], dtype={'codigo_producto': str})
            print(f" - Dataset '{nombre}' cargado. (Dimensiones: {datasets_para_modelar[nombre].shape})")
        except FileNotFoundError:
            print(f" - ADVERTENCIA: No se encontró el archivo para '{nombre}' en {path}. Se omitirá.")

    if not datasets_para_modelar:
        print(f"ADVERTENCIA: No hay datasets cargados para '{target}'. Saltando modelado.")
        continue
        
    # --- Ejecución de Experimentos (¡Llamada actualizada!) ---
    # Ahora pasamos 'maxiter' a la función
    logging.info("Iniciando la ejecución de experimentos de modelado (con CV)...")
    resultados_completos = ejecutar_experimentos(
        datasets_dict=datasets_para_modelar,
        df_info_productos=df_info_productos,
        target_column=target,
        models_to_run=models_to_run, 
        metrics_to_calculate=metrics_to_calculate,
        maxiter_sarima=maxiter # <-- ¡NUEVO PARÁMETRO PASADO!
    )

    # --- Guardado de Resultados (¡Ordenamiento mejorado!) ---
    print(f"\nGuardando resultados detallados (CV) para '{target}'...")
    if resultados_completos:
        for nombre, df_res in resultados_completos.items():
            if df_res.empty:
                print(f" - Sin resultados para '{nombre}'.")
                continue
                
            path = os.path.join(output_subdir, f'resultados_detallados_{nombre}_{target}_CV.csv') # Añadido _CV al nombre
            
            # --- ¡LÓGICA DE ORDENAMIENTO MEJORADA! ---
            # Ordenar por MASE (ascendente) como métrica principal para la tesis
            sort_col = 'MASE_ganador' # Esta columna ahora es el MASE_CV_mean del ganador
            ascending_sort = True
            
            if sort_col not in df_res.columns:
                # Fallback si MASE no se calculó
                sort_col = 'R2_ganador'
                ascending_sort = False
            
            try:
                if sort_col in df_res.columns:
                     df_res.sort_values(by=sort_col, ascending=ascending_sort).to_csv(path, index=False, decimal='.', sep=',')
                else:
                     df_res.to_csv(path, index=False, decimal='.', sep=',') # Guardar sin ordenar si no hay métricas
                print(f" -> Resultados para '{nombre}' guardados en: {path}")
            except Exception as e:
                print(f"ERROR al guardar {path}: {e}")
    else:
        print("\nNo se generaron resultados. Verifica los pasos anteriores.") 

print(f"\n{'='*20} PIPELINE DE MODELADO (CON CV) COMPLETADO {'='*20}\n")


KeyboardInterrupt: 